# initial clean data of Bicicletar datasets
## to-do list
 - load the datasets
 - rename column names
 - combine datetime columns
 - remove columns with duplication information
 - remove rows with jornadas < t_limit

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import re

In [27]:
def my_func(row):
    mat=re.match('(\d{2})[/](\d{2})[/](\d{4})[ ](\d{2})[:](\d{2})[:](\d{2})$', str(row))
    if mat is not None:
        return datetime.datetime.strptime(row, "%d/%m/%Y %H:%M:%S").strftime('%Y')
    else:
        return datetime.datetime.strptime(str(row), "%Y-%m-%d %H:%M:%S").strftime('%Y')

def my_func2(a,b):
    b0, b1 = b.split(' ')
    horas=int(re.sub("\D", "", b0))
    minutos=int(re.sub("\D", "", b1))
    return a + timedelta(minutes=minutos, hours=horas)

#---------------------------------------------------------------------------------------------------------------#
    
def clear_data(year):    
    # load the data
    brute_path = '../data/bicicletar/'+str(year)+'.xlsx'    
    brute_data = pd.read_excel(brute_path)
    
    if (year != 2017): 
        # change the columns name
        clean_data = brute_data.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                           'IdUsuario': 'id_usuario',
                                                           'AnoNascimento': 'ano_nascimento',
                                                           'Sexo': 'sexo',
                                                           'Pais': 'pais',
                                                           'Distrito': 'distrito',
                                                           'Cidade': 'cidade',
                                                           'UF': 'uf',
                                                           'DataCadastro': 'data_cadastro',
                                                           'Meio de contato para retirada': 'tipo_usuario',
                                                           'Bicicleta': 'id_bicicleta',
                                                           'DataRetirada': 'data_retirada',
                                                           'HoraRetirada': 'hora_retirada',
                                                           'DataDevolucao': 'data_devolucao',
                                                           'HoraDevolucao': 'hora_devolucao',
                                                           'EstacaoRetirada': 'estacao_retirada',
                                                           'EstacaoDevolucao': 'estacao_devolucao'});
        # set id_jornada to index
        clean_data = clean_data.set_index('id_jornada')

        # change type of data_cadastro
        clean_data['data_cadastro'] = clean_data['data_cadastro'].astype('datetime64')

        # combine data_retirada with hora_retirada
        clean_data['datetime_retirada'] = clean_data['data_retirada'] + pd.to_timedelta(clean_data['hora_retirada'])
        
        # combine data_devolucao with hora_devolucao
        clean_data['datetime_devolucao'] = clean_data['data_devolucao'] + pd.to_timedelta(clean_data['hora_devolucao'])

        # drop columns with duplicate information columns
        clean_data = clean_data.drop(['TempoJornada',
                                                'TempoJornadaMinutos',
                                                'data_retirada',
                                                'hora_retirada',
                                                'data_devolucao',
                                                'hora_devolucao',
                                                'distrito'], axis=1)
        
    else:       
        # change the columns name
        clean_data = brute_data.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                           'globalId': 'id_usuario',
                                                           'Sexo': 'sexo',
                                                           'País': 'pais',
                                                           'Cidade': 'cidade',
                                                           'UF': 'uf',
                                                           'Data de Cadastro': 'data_cadastro',
                                                           'Meio de Retirada': 'tipo_usuario',
                                                           'NumExterno': 'id_bicicleta',
                                                           'DataCorrida': 'data_corrida',
                                                           'HoraRetirada': 'hora_retirada',
                                                           'HoraDevolucao': 'hora_devolucao',
                                                           'EstacaoRetirada': 'estacao_retirada',
                                                           'EstacaoDevolucao': 'estacao_devolucao'});

        # set index column
        clean_data = clean_data.set_index('id_jornada')

        # merge data_corrida with hora_retirada
        clean_data['datetime_retirada'] = clean_data['data_corrida'] + pd.to_timedelta(clean_data['hora_retirada'].astype(str))

        # merge data_corrida with hora_devolucao
        clean_data['datetime_devolucao'] = clean_data.apply(lambda row: my_func2(row['data_corrida'],row['Duração da Corrida']), axis=1)
        
        # fix wrong date inputs
        brute_data['ano_nascimento'] = brute_data.apply(lambda row: my_func(row['Nascimento']), axis=1)
        brute_data['ano_nascimento'].astype('int')
        
        clean_data['datetime_devolucao'] = clean_data['data_corrida'] + pd.to_timedelta(clean_data['hora_devolucao'].astype(str))

        # drop columns with duplicate information columns
        clean_data = clean_data.drop(['Nascimento',
                                      'Projeto',
                                      'DiaSemana',
                                      'AreaEstacaoRetirada',
                                      'EnderecoEstacaoRetirada',
                                      'AreaEstacaoDevolucao',
                                      'EnderecoEstacaoDevolucao',
                                      'Duração da Corrida',
                                      'data_corrida',
                                      'hora_retirada',
                                      'hora_devolucao'], axis=1)
    filename = 'clean_data_' + str(year) + '.csv'
    clean_data.index.names = ['id_ano']
    clean_data.to_csv('../data/bicicletar/'+filename)
    print('## file \''+filename+'\' created.')
    

In [28]:
# clear_data(2015)
# clear_data(2016)
clear_data(2017)

## file 'clean_data_2017.csv' created.


In [48]:
datapath = '../data/bicicletar/clean_data_'
# data2015 = pd.read_csv(datapath+'2015.csv')
# data2016 = pd.read_csv(datapath+'2016.csv')
data2017 = pd.read_csv(datapath+'2017.csv')

# all_data = pd.concat([data2015,data2016,data2017])
all_data = data2017
# all_data.index.names = ['id_geral']


# print('2015 ',data2015.shape)
# print('2016 ',data2016.shape)
print('2017 ',data2017.shape)

2017  (491552, 13)


In [5]:
'''
~> A função 'dataInfo' levanta algumas informações:
  ::Outliers
     1. Qual menor tempo de uma viagem entre 2 estações diferentes?
     2. Menor e maior tempo de viagem?
     3. 
  
  ::Minibicicletar
     1. Quantas viagens?
     2. Idade desses usuários

'''
import matplotlib.dates as mdates
from datetime import timedelta

def dataInfo(data):
    #calcula diferenca de tempo de devoluçao e retirada
    datasRet = pd.to_datetime(data['datetime_retirada'])
    datasDev = pd.to_datetime(data['datetime_devolucao'])
    dif = datasDev - datasRet    
    
    
    # Seleciona viagens com Estação Final é diferente da Estação Inicial
    difStations = data[data['estacao_retirada'] != data['estacao_devolucao']]
    DS_datasRet = pd.to_datetime(difStations['datetime_retirada'])
    DS_datasDev = pd.to_datetime(difStations['datetime_devolucao'])
    
    #print(difStations)
    DS_dif = DS_datasDev - DS_datasRet    
    return data[dif<timedelta(minutes=0)][['datetime_retirada','datetime_devolucao','id_ano']], dif
#     print(dif<timedelta(minutes=0))
#     return dif
#     DS_dif[DS_dif>timedelta(minutes=0)]
#     print(DS_dif[DS_dif>timedelta(days=1)].shape[0])
    
    
    #print(difStations.loc[(DS_dif>0)])
                       
    #print(DS_dif.min(),DS_dif.max()) 
    #print(DS_dif.sort_values(ascending=True))
    
       

In [32]:
'''this function excludes a few rides based on the following criteria:
   1. Starts and finish on the same station;
   2. Its total time is less than < t; t is to be defined.
'''

from datetime import timedelta

def remove_rides(data,printReport):
    datasRet = pd.to_datetime(data['datetime_retirada'])
    datasDev = pd.to_datetime(data['datetime_devolucao'])
    dif = datasDev - datasRet
    t = [2,3,5,7,9]    
    for i in t:
        min = timedelta(minutes=i)         
        selectedData = data.loc[(dif>=min)]
        removedData = data.loc[(dif<min)]
        selectedData2 = removedData[removedData['estacao_retirada'] != removedData['estacao_devolucao']]
        
        if printReport:
            print('Minutos:',i)
            print('Viagens com tempo acima do mínimo:',selectedData.shape[0])                
            print('Viagens com tempo abaixo do mínimo:',removedData.shape[0])               
            print('Viagens Recuperadas',selectedData2.shape[0])        
            print('TOTAl:',(selectedData.shape[0]+selectedData2.shape[0]))        

    print('########################################################')
    print("Threshold selecionado: 3 minutos;") 
    
    min = timedelta(minutes=3)         
    selectedData = data.loc[(dif>=min)]        
    removedData = data.loc[(dif<min)]
    selectedData2 = removedData[removedData['estacao_retirada'] != removedData['estacao_devolucao']]
    finalCleanData = pd.concat([selectedData,selectedData2])
        
    print('TOTAl de viagens selecionadas: %d de %d (%.2f%%)' %(finalCleanData.shape[0],data.shape[0],(finalCleanData.shape[0]/data.shape[0])*100))    
    return selectedData
           

In [33]:
all_data = remove_rides(all_data,False)

########################################################
Threshold selecionado: 3 minutos;
TOTAl de viagens selecionadas: 1216446 de 1344591 (90.47%)


In [ ]:
print('Distribuição por Ano de nascimento: ')
perYear = all_data.groupby(by='ano_nascimento', as_index=True).agg({'id_usuario': pd.Series.nunique})
perYear.plot.bar()

In [ ]:
uniques = all_data.id_usuario.unique()
print('Total de usuários únicos: ',len(uniques))
print('Distribuição por sexo: ')
perSex = all_data.groupby(by='sexo', as_index=True).agg({'id_usuario': pd.Series.nunique})
perSex.plot.bar()


In [ ]:

print('Distribuição num de viagens')
ridePerId = all_data.groupby(by='id_usuario', as_index=True).agg({'id_jornada': pd.Series.nunique})


In [35]:
data_, diff = dataInfo(all_data)




In [50]:
idxBugs = data_.assign(diff=diff[diff<timedelta(minutes=0)]).sort_values(by='id_geral').index
# print(idxBugs.shape)
all_data.loc[idxBugs]


/home/renan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


,id_ano,id_usuario,sexo,pais,cidade,uf,data_cadastro,estacao_retirada,tipo_usuario,estacao_devolucao,id_bicicleta,datetime_retirada,datetime_devolucao
id_geral,,,,,,,,,,,,,
392,3283998.0,1683482.0,M,BR,Fortaleza,CE,27/12/2014 16:55:59,25 - Torres Câmar,Cartão Transportes,63 - Júlio Azeved,15979.0,2017-01-01 17:47:06,2017-01-01 08:16:31
616,3284256.0,2470467.0,M,BR,NaN,CE,24/12/2015 03:49:24,53 - Extra Aguanamb,Cartão Transportes,70 - Shopping Iguatem,15677.0,2017-01-01 20:16:15,2017-01-01 10:17:45
745,3284398.0,1785713.0,F,BR,fortaleza,CE,24/01/2015 01:24:56,1 - Praça Luíza Távor,Cartão Transportes,14 - Aterro Praia de Iracem,10493.0,2017-01-01 23:35:10,2017-01-01 00:23:58
746,3284399.0,2871003.0,M,BR,NaN,CE,17/06/2016 09:27:29,20 - Dragão do Ma,Cartão Transportes,12 - Ana Bilha,14632.0,2017-01-01 23:36:42,2017-01-01 00:50:38
1345,3285116.0,3510278.0,F,BR,NaN,SP,2017-02-01 16:39:43,14 - Aterro Praia de Iracem,URA/APP,14 - Aterro Praia de Iracem,14620.0,2017-01-02 15:41:46,2017-01-02 08:26:07
2004,3285915.0,2178948.0,M,BR,fortaleza,CE,26/07/2015 10:21:22,38 - Instituto dos Cego,URA/APP,32 - Praça da Gentilândi,14855.0,2017-01-02 19:11:35,2017-01-02 08:37:23
2144,3286095.0,1919640.0,M,BR,fortaleza,CE,23/03/2015 15:42:24,79 - Liceu do Cear,Cartão Transportes,79 - Liceu do Cear,15260.0,2017-01-02 20:08:57,2017-01-02 17:04:05
2254,3286221.0,3485526.0,F,BR,NaN,RO,26/12/2016 14:54:29,13 - Aterrinho da Praia de Iracem,URA/APP,13 - Aterrinho da Praia de Iracem,15961.0,2017-01-02 21:02:30,2017-01-02 16:17:21
2356,3286345.0,3507580.0,M,BR,NaN,NaN,2017-01-01 22:34:47,14 - Aterro Praia de Iracem,URA/APP,14 - Aterro Praia de Iracem,14584.0,2017-01-02 23:20:26,2017-01-02 09:09:13


In [51]:
all_data

,id_ano,id_usuario,sexo,pais,cidade,uf,data_cadastro,estacao_retirada,tipo_usuario,estacao_devolucao,id_bicicleta,datetime_retirada,datetime_devolucao
0,3283524,1884685,F,BR,Fortaleza,CE,2015-05-03 15:15:46,13 - Aterrinho da Praia de Iracem,Cartão Transportes,38 - Instituto dos Cego,14572,2017-01-01 05:00:39,2017-01-01 06:03:49
1,3283525,2272501,M,BR,Fortaleza,CE,2015-09-09 10:32:08,13 - Aterrinho da Praia de Iracem,Cartão Transportes,38 - Instituto dos Cego,15988,2017-01-01 05:01:17,2017-01-01 06:04:22
2,3283526,3173880,M,BR,NaN,CE,16/09/2016 16:30:30,13 - Aterrinho da Praia de Iracem,Cartão Transportes,77 - Faculdade Lourenço Filh,15675,2017-01-01 05:01:31,2017-01-01 05:17:33
3,3283527,1016301,F,BR,Belém,PA,24/02/2014 13:29:43,14 - Aterro Praia de Iracem,URA/APP,14 - Aterro Praia de Iracem,15274,2017-01-01 05:03:38,2017-01-01 05:50:30
4,3283528,3448123,M,BR,NaN,NaN,15/12/2016 14:25:28,60 - Salesiano Dom Bosc,Cartão Transportes,52 - Esplanada Montes,15958,2017-01-01 05:03:55,2017-01-01 05:26:00
5,3283529,2216307,M,BR,Fortaleza,CE,2015-12-08 15:13:20,20 - Dragão do Ma,Cartão Transportes,49 - Costa Mende,14629,2017-01-01 05:06:22,2017-01-01 05:34:13
6,3283530,2760401,M,BR,NaN,CE,19/04/2016 22:28:24,9 - Moreira da Roch,Cartão Transportes,77 - Faculdade Lourenço Filh,11565,2017-01-01 05:12:15,2017-01-01 05:33:39
7,3283531,3258017,M,BR,NaN,CE,21/10/2016 18:40:16,9 - Moreira da Roch,Cartão Transportes,77 - Faculdade Lourenço Filh,15640,2017-01-01 05:12:36,2017-01-01 05:33:37
8,3283532,1806765,M,BR,NaN,CE,30/01/2015 21:52:32,17 - BNB Cultural Catedra,URA/APP,1 - Praça Luíza Távor,10607,2017-01-01 05:27:12,2017-01-01 05:41:43
9,3283534,1634717,F,BR,FORTALEZA,CE,13/12/2014 05:48:48,9 - Moreira da Roch,Cartão Transportes,12 - Ana Bilha,11574,2017-01-01 05:31:55,2017-01-01 05:40:08


In [64]:
sample = all_data.loc[392]['datetime_retirada']
DM = timedelta(minutes=10,hours=20)
print(DM,sample)
DM+pd.to_datetime(sample)

20:10:00 2017-01-01 17:47:06


Timestamp('2017-01-02 13:57:06')

In [63]:
all_data.dtypes

id_ano                 int64
id_usuario             int64
sexo                  object
pais                  object
cidade                object
uf                    object
data_cadastro         object
estacao_retirada      object
tipo_usuario          object
estacao_devolucao     object
id_bicicleta           int64
datetime_retirada     object
datetime_devolucao    object
dtype: object